##Decision Tree Classifier - Optimized

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_json("train.json")
train_df.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [26]:
test_df = pd.read_json("test.json")
test_df.head()

,id,ingredients
0,18009,"[baking powder, eggs, all-purpose flour, raisi..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta..."
2,41580,"[sausage links, fennel bulb, fronds, olive oil..."
3,29752,"[meat cuts, file powder, smoked sausage, okra,..."
4,35687,"[ground black pepper, salt, sausage casings, l..."


In [16]:
#list of all possible ingredients

ingredients_all=[]

for i in train_df['ingredients']:
    for j in xrange(len(i)):
        ingredients_all.append(i[j])
        
for i in test_df['ingredients']:
    for j in xrange(len(i)):
        ingredients_all.append(i[j])

features=np.unique(ingredients_all) #create a feature list of unique ingrediants.

ingredients_map={v:k for k,v in enumerate(np.unique(ingredients_all).tolist())} #assign key to each ingredient

In [5]:
def build_designmatrix(indf):
    
    designmatrix=np.zeros((len(indf), len(features))) 
    all_ids = indf['id'] #list of all id's from input set.
    
    #build design matrix
    for i in xrange(len(indf)):
        for j in indf['ingredients'][indf['id']==all_ids[i]]:
                for k in xrange(len(j)):
                        designmatrix[i, ingredients_map[j[k]]] = 1
                        
    return designmatrix

In [6]:
%%time
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(build_designmatrix(train_df), train_df['cuisine'])
clf_predict=clf.predict(build_designmatrix(test_df))

CPU times: user 7min 3s, sys: 7.37 s, total: 7min 10s
Wall time: 7min 17s


In [12]:
predictions_as_dataframe = test_df.join(pd.DataFrame({"Prediction DT": clf_predict}))
print predictions_as_dataframe

         id                                        ingredients Prediction DT
0     18009  [baking powder, eggs, all-purpose flour, raisi...        french
1     28583  [sugar, egg yolks, corn starch, cream of tarta...   southern_us
2     41580  [sausage links, fennel bulb, fronds, olive oil...       italian
3     29752  [meat cuts, file powder, smoked sausage, okra,...  cajun_creole
4     35687  [ground black pepper, salt, sausage casings, l...       italian
5     38527  [baking powder, all-purpose flour, peach slice...   southern_us
6     19666             [grape juice, orange, white zinfandel]       italian
7     41217  [ground ginger, white pepper, green onions, or...       chinese
8     28753  [diced onions, taco seasoning mix, all-purpose...       mexican
9     22659  [eggs, cherries, dates, dark muscovado sugar, ...       british
10    21749  [pasta, olive oil, crushed red pepper, cherry ...       italian
11    44967  [water, butter, ground sumac, ground lamb, gro...         greek

In [35]:
%%time
from sklearn import preprocessing
from sklearn.linear_model import Ridge
est = Ridge(alpha=10)

labels = train_df['cuisine']
le = preprocessing.LabelEncoder()
labels_fea = le.fit_transform(labels)
est_fit=est.fit(build_designmatrix(train_df), labels_fea)
est_predict=est_fit.predict(build_designmatrix(test_df))
labels_converted = le.inverse_transform(est_predict.astype('I'))



IndexError: index 28 is out of bounds for axis 0 with size 20

In [32]:
predictions_as_dataframe = test_df.join(pd.DataFrame({"Prediction R": labels_converted}))
print predictions_as_dataframe

ValueError: LabelEncoder was not fitted yet.